This is the Graph Neural Network model!

## 1. Set device to GPU

Click `Runtime` and then under `Change runtime type` set `hardware accelerator` to GPU.

## 2. Import necessary packages

Import `torch` and `os`.

In [ ]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 2.0.1+cu118


Import `torch-scatter` and `torch-sparse`, which are packages necessary for `torch-geometric` (PyTorch Geometric, the library for graph machine learning). Also import `ogb`, the Open Graph Benchmark library, to access graph datasets.

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-geometric
!pip install ogb

from IPython.display import clear_output
clear_output()

## 3. Read in loaded and proprocessed dataset (TODO)

In [ ]:
import pandas as pd
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator

/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_scatter_cuda.so: undefined symbol: _ZN2at4_ops6narrow4callERKNS_6TensorElll
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_sparse/_diag_cuda.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


The `make_symmetric_sparse` method takes a sparse tensor and makes it symmetric.

In [ ]:
import torch

def make_symmetric_sparse(tensor):
    # Extract the row, column, and value data from the sparse tensor
    row = tensor.crow_indices()
    col = tensor.col_indices()
    value = tensor.values()

    # Create a symmetric index array by stacking row and column indices in reverse order
    sym_row = torch.cat([row, col], dim=0)
    sym_col = torch.cat([col, row], dim=0)

    # Combine the value tensor with itself to get the symmetric values
    sym_value = torch.cat([value, value], dim=0)

    # Check for duplicates in the symmetric index array
    sym_indices = torch.stack([sym_row, sym_col], dim=0)
    _, unique_idx = torch.unique(sym_indices, dim=1, return_inverse=True)
    unique_idx = unique_idx.view(-1)

    # Remove duplicates from the symmetric index array
    sym_row = sym_row[unique_idx]
    sym_col = sym_col[unique_idx]
    sym_value = sym_value[unique_idx]

    # Create the symmetric sparse tensor
    symmetric_tensor = torch.sparse_coo_tensor(
        indices=torch.stack([sym_row, sym_col], dim=0),
        values=sym_value,
        size=tensor.size(),
    )

    return symmetric_tensor


For now, we're using a graph and datasets provided by `ogb`. Eventualy, we'll create our own graph and datasets using the airline datasets.

In [ ]:
dataset_name = 'ogbl-arxiv'
dataset = PygLinkPropPredDataset(name=dataset_name,
                                  transform=T.ToSparseTensor())

data = dataset[0]

a = data.adj_t

print(a)
print(type(a))
print(a.crow_indices())
print(a.col_indices())
print(a.values())
print(a.crow_indices().shape)
print(a.col_indices().shape)
print(a.values().shape)

# symmetric_tensor = make_symmetric_sparse(a)
# data.adj_t = symmetric_tensor
# data.adj_t = data.adj_t.coalesce()

'''
n = a.shape[0]
for i in range(n):
  for j in range(n):
    if a[i, j] != a[j, i]:
      print(i, j, a[i, j], a[j, i])
'''

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

tensor(crow_indices=tensor([      0,     289,     290,  ..., 1166240,
                            1166243, 1166243]),
       col_indices=tensor([   411,    640,   1162,  ...,  30351,  35711,
                           103121]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]), size=(169343, 169343),
       nnz=1166243, layout=torch.sparse_csr)
<class 'torch.Tensor'>
tensor([      0,     289,     290,  ..., 1166240, 1166243, 1166243])
tensor([   411,    640,   1162,  ...,  30351,  35711, 103121])
tensor([1., 1., 1.,  ..., 1., 1., 1.])
torch.Size([169344])
torch.Size([1166243])
torch.Size([1166243])
Device: cuda


## 4. Create Models

For now, we can stick with GCN (Graph Convolutional Network) and GAT (Graph Attention Network). May implement GIN in the future, but it's more complicated.

In [ ]:
from torch_geometric.nn import GCNConv, GINConv, GATConv

### GCN

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GCN, self).__init__()

        # A list of GCNConv layers
        self.convs = torch.nn.ModuleList(
            [GCNConv(in_channels=input_dim, out_channels=hidden_dim)] +
            [GCNConv(in_channels=hidden_dim, out_channels=hidden_dim)
                for i in range(num_layers-2)] +
            [GCNConv(in_channels=hidden_dim, out_channels=output_dim)]
        )

        # A list of 1D batch normalization layers
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(num_features=hidden_dim)
                for i in range(num_layers-1)
        ])


        # The log softmax layer
        self.softmax = torch.nn.LogSoftmax()

        # Probability of an element to be zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for conv, bn in zip(self.convs[:-1], self.bns):
            x1 = F.relu(bn(conv(x, adj_t)))
            if self.training:
                x1 = F.dropout(x1, p=self.dropout)
            x = x1
        x = self.convs[-1](x, adj_t)
        out = x if self.return_embeds else self.softmax(x)

        return out

### GIN

In [ ]:
class GIN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GIN, self).__init__()

        # A list of GINConv layers
        self.convs = torch.nn.ModuleList(
            [GINConv(in_channels=input_dim, out_channels=hidden_dim)] +
            [GINConv(in_channels=hidden_dim, out_channels=hidden_dim)
                for i in range(num_layers-2)] +
            [GINConv(in_channels=hidden_dim, out_channels=output_dim)]
        )

        # A list of 1D batch normalization layers
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(num_features=hidden_dim)
                for i in range(num_layers-1)
        ])


        # The log softmax layer
        self.softmax = torch.nn.LogSoftmax()

        # Probability of an element to be zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for conv, bn in zip(self.convs[:-1], self.bns):
            x1 = F.relu(bn(conv(x, adj_t)))
            if self.training:
                x1 = F.dropout(x1, p=self.dropout)
            x = x1
        x = self.convs[-1](x, adj_t)
        out = x if self.return_embeds else self.softmax(x)

        return out

### GAT

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GAT, self).__init__()

        # A list of GINConv layers
        self.convs = torch.nn.ModuleList(
            [GATConv(in_channels=input_dim, out_channels=hidden_dim)] +
            [GATConv(in_channels=hidden_dim, out_channels=hidden_dim)
                for i in range(num_layers-2)] +
            [GATConv(in_channels=hidden_dim, out_channels=output_dim)]
        )

        # A list of 1D batch normalization layers
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(num_features=hidden_dim)
                for i in range(num_layers-1)
        ])


        # The log softmax layer
        self.softmax = torch.nn.LogSoftmax()

        # Probability of an element to be zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for conv, bn in zip(self.convs[:-1], self.bns):
            x1 = F.relu(bn(conv(x, adj_t)))
            if self.training:
                x1 = F.dropout(x1, p=self.dropout)
            x = x1
        x = self.convs[-1](x, adj_t)
        out = x if self.return_embeds else self.softmax(x)

        return out

### GAT and GCN Hybrid

In [ ]:
class GATC(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GATC, self).__init__()

        # A list of GINConv layers
        self.convs = torch.nn.ModuleList(
            [GATConv(in_channels=input_dim, out_channels=hidden_dim)] +
            [GATConv(in_channels=hidden_dim, out_channels=hidden_dim)
                for i in range(num_layers-3)] +
            [GCNConv(in_channels=hidden_dim, out_channels=hidden_dim)]
        )

        # A list of 1D batch normalization layers
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(num_features=hidden_dim)
                for i in range(num_layers-1)
        ])

        self.lstm = torch.nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim)


        # The log softmax layer
        self.softmax = torch.nn.LogSoftmax()

        # Probability of an element to be zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        x = self.lstm(x)
        for conv, bn in zip(self.convs[:-1], self.bns):
            x1 = F.relu(bn(conv(x, adj_t)))
            if self.training:
                x1 = F.dropout(x1, p=self.dropout)
            x = x1
        x = self.convs[-1](x, adj_t)

        out = x if self.return_embeds else self.softmax(x)

        return out

## 5. Define the `train` and `test` functions

In [ ]:
def train(model, data, train_idx, optimizer, loss_fn):
    model.train()
    loss = 0

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    loss = loss_fn(out[train_idx], data.y[train_idx].reshape(-1))

    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    model.eval()

    # The output of model on all data
    out = None

    out = model(data.x, data.adj_t)

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      # Save locally as csv
      df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

## 6. Set arguments for model

In [ ]:
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 128,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}

## 7. Initialize model and evaluator

Choose the model type from among `GCN`, `GAT`, `GIN`, and `GATC`:

In [ ]:
model_type = GAT

model = model_type(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name=dataset_name)

## 8. Train the model

In [ ]:
import copy

# reset the parameters to initial random value
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, optimizer, loss_fn)
  result = test(model, data, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

OutOfMemoryError: ignored

## 9. Get the best model

In [ ]:
best_result = test(best_model, data, split_idx, evaluator, save_model_results=True)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')